# Problem 7.1

In [3]:
import glob
import os
import warnings
import time

# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy.signal
import scipy.stats as st

# BE/Bi 103 utilities
import bebi103

# Image processing tools
import skimage
import skimage.io

# Import plotting tools
import matplotlib.pyplot as plt
import seaborn as sns

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

# This enables high res graphics inline (only use with static plots (non-Bokeh))
# SVG is preferred, but there is a bug in Jupyter with vertical lines
%config InlineBackend.figure_formats = {'png', 'retina'}

# JB's favorite Seaborn settings for notebooks
rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)

# Suppress future warnings
warnings.filterwarnings('ignore')

In [4]:
# The directory containing daytime data
data_dir = '/Users/elenaperry/Dropbox/Caltech/bebi103/Cassiopea_Pulsation/day'

# Glob string for images
im_glob = os.path.join(data_dir, '*.TIF')

In [5]:
def squish_rgb(fname):
    return skimage.io.imread(fname)[0,:,:]

ic = skimage.io.ImageCollection(im_glob, conserve_memory=True, 
                                load_func=squish_rgb)

In [6]:
# We used plt.ginput(100) to select vertices for each jellyfish's ROI
verts1 = np.array([(35.863636363636388, 107.22727272727269),
 (173.59090909090912, 111.31818181818176),
 (168.13636363636368, 223.13636363636363),
 (34.500000000000014, 216.31818181818181)])
 
verts2 = np.array([(185.86363636363643, 100.40909090909088),
 (338.59090909090918, 111.31818181818176),
 (330.40909090909099, 232.68181818181813),
 (185.86363636363643, 224.5)])
 
verts3 = np.array([(349.50000000000006, 99.045454545454504),
 (498.13636363636374, 100.40909090909088),
 (495.40909090909099, 236.77272727272725),
 (348.13636363636368, 238.13636363636363)])
 
verts4 = np.array([(504.9545454545455, 90.863636363636317),
 (635.86363636363649, 97.68181818181813),
 (634.50000000000011, 244.95454545454544),
 (506.31818181818187, 239.5)])

verts5 = np.array([(38.590909090909108, 232.68181818181813),
 (173.59090909090912, 239.5),
 (169.50000000000006, 379.95454545454544),
 (34.500000000000014, 363.59090909090907)])

verts6 = np.array([(184.50000000000006, 250.40909090909088),
 (335.86363636363643, 253.13636363636363),
 (331.77272727272731, 392.22727272727275),
 (177.68181818181824, 386.77272727272725)])

verts7 = np.array([(345.40909090909099, 249.0454545454545),
 (492.68181818181824, 250.40909090909088),
 (485.86363636363649, 393.59090909090907),
 (345.40909090909099, 389.5)])

verts8 = np.array([(502.22727272727275, 250.40909090909088),
 (633.13636363636374, 255.86363636363632),
 (633.13636363636374, 393.59090909090907),
 (503.59090909090912, 393.59090909090907)])

all_verts = np.array([verts1, verts2, verts3, verts4,
                  verts5, verts6, verts7, verts8])

In [7]:
def local_min(x, y):
    """
    Fit three points to quadratic and return location of maximum or minimum.
    """
    a, b, c = np.polyfit(x, y, 2)
    x_max = -b / 2 / a
    return x_max, a*x_max**2 + b*x_max + c

In [ ]:
# Define load_func that only gives region bounding ROI
    def load_roi(fname, roi_bbox=None):
        if roi_bbox is None:
            return skimage.io.imread(fname)[0,:,:]
        else:
            return skimage.io.imread(fname)[0,:,:][roi_bbox]

    # Load image collection
    ic = skimage.io.ImageCollection(im_glob, conserve_memory=True, 
                                    load_func=load_roi, roi_bbox=roi_bbox)

In [8]:
def intercontraction_times(verts, ic_shape):
    
    # Define ROI
    roi, roi_bbox, roi_box = bebi103.verts_to_roi(verts, *ic_shape)
    
    # Generate timestamps for the images
    fps = 15
    t = np.arange(0, len(ic)) / fps

    # Set up NumPy array to store total pixel intensity
    total_int = np.empty(len(t))

    # Look through and compute total intensity in the ROI
    for i, im in enumerate(ic):
        total_int[i] = ic[i][roi_box].sum()

    # Subtract the mean and rescale the data to go from -1 to 1
    total_int -= total_int.mean()
    total_int = 1 + 2 / (total_int.max() - total_int.min()) \
                    * (total_int - total_int.max())

    # Find up and down crossing indices
    up_crossing_inds = np.where(
        np.logical_and(total_int[:-1] < 0, total_int[1:] >= 0))[0]
    down_crossing_inds = np.where(
        np.logical_and(total_int[:-1] > 0, total_int[1:] <= 0))[0] + 1

    # Make sure upcrossing are first
    if down_crossing_inds[0] < up_crossing_inds[0]:
        down_crossing_inds = down_crossing_inds[1:]

    # Make sure downcrossing last
    if up_crossing_inds[-1] > down_crossing_inds[-1]:
        up_crossing_inds = up_crossing_inds[:-1]

    # Find maxima
    t_peaks = np.empty(len(up_crossing_inds))
    peaks = np.empty_like(t_peaks)
    for i, uc in enumerate(up_crossing_inds):
        ind = uc + np.argmax(total_int[uc:down_crossing_inds[i]+1])
        t_peaks[i], peaks[i] = local_min(t[ind-1:ind+2], total_int[ind-1:ind+2])

    # Intercontractions times
    t_int = np.diff(t_peaks)
    
    return t, total_int, t_peaks, peaks, t_int

In [9]:
def intercontraction_times2(verts, ic_shape):
    
    # Define ROI
    roi, roi_bbox, roi_box = bebi103.verts_to_roi(verts, *ic_shape)

    # Generate timestamps for the images
    fps = 15
    t = np.arange(0, len(ic)) / fps

    # Set up NumPy array to store total pixel intensity
    total_int = np.empty(len(t))

    # Look through and compute total intensity in the ROI
    for i, im in enumerate(ic):
        total_int[i] = ic[i][roi].sum()

    # Subtract the mean and rescale the data to go from -1 to 1
    total_int -= total_int.mean()
    total_int = 1 + 2 / (total_int.max() - total_int.min()) \
                    * (total_int - total_int.max())

    # Find up and down crossing indices
    up_crossing_inds = np.where(
        np.logical_and(total_int[:-1] < 0, total_int[1:] >= 0))[0]
    down_crossing_inds = np.where(
        np.logical_and(total_int[:-1] > 0, total_int[1:] <= 0))[0] + 1

    # Make sure upcrossing are first
    if down_crossing_inds[0] < up_crossing_inds[0]:
        down_crossing_inds = down_crossing_inds[1:]

    # Make sure downcrossing last
    if up_crossing_inds[-1] > down_crossing_inds[-1]:
        up_crossing_inds = up_crossing_inds[:-1]

    # Find maxima
    t_peaks = np.empty(len(up_crossing_inds))
    peaks = np.empty_like(t_peaks)
    for i, uc in enumerate(up_crossing_inds):
        ind = uc + np.argmax(total_int[uc:down_crossing_inds[i]+1])
        t_peaks[i], peaks[i] = local_min(t[ind-1:ind+2], total_int[ind-1:ind+2])

    # Intercontractions times
    t_int = np.diff(t_peaks)
    
    return t, total_int, t_peaks, peaks, t_int

In [ ]:
start = time.time()
t1, total_int1, t_peaks1, peaks1, t_int1 = intercontraction_times(all_verts[0], ic[0].shape)
print('The first function took ' + str(time.time()-start) + ' seconds')


In [ ]:
start = time.time()
t2, total_int2, t_peaks2, peaks2, t_int2 = intercontraction_times2(all_verts[0], ic[0].shape)
print('The second function took ' + str(time.time()-start) + ' seconds')